### 8.2 Customizing string formatting

In [1]:
_formats = {
    'ymd': '{d.year}-{d.month}-{d.day}',
    'mdy': '{d.month}/{d.day}/{d.year}',
    'dmy': '{d.day}/{d.month}/{d.year}'
}

In [2]:
class Date:

    def __init__(self, day, month, year):
        self.day , self.month, self.year = day, month, year
        
    def __format__(self, code):
        if code=='':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [3]:
d = Date(31, 7, 1985)
print(format(d, 'dmy'))
print(format(d, 'mdy'))
print('{:ymd}'.format(d))

31/7/1985
7/31/1985
1985-7-31


### 8.8 Extending a Property in a Subclass

In [4]:
class Person:
    
    def __init__(self, name):
        self.name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string.')
        self._name = value

In [5]:
class SubPerson(Person):
    
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    
    @name.setter
    def name(self, value):
        print('Setting name')
        # https://stackoverflow.com/questions/38661438/python-super-two-argument-version-in-context-of-new
        super(SubPerson, SubPerson).name.__set__(self, value) # Having a class as the second argument is to access the __set__ class method

In [6]:
s = Person('Guido')
print(s.name)
s.name = 'BDFL'
print(s.name)

Guido
BDFL


In [7]:
l = SubPerson('Larry')
print(l.name)
l.name = 'Larry Wall'
print(l.name)

Setting name
Getting name
Larry
Setting name
Getting name
Larry Wall


In [8]:
class AnotherPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name for AnotherPerson')
        return super().name
        

In [9]:
c = AnotherPerson('Bjarne')
print(c.name)
c.name += ' Stroustrop' # Will call the getter again
print(c.name)

Getting name for AnotherPerson
Bjarne
Getting name for AnotherPerson
Getting name for AnotherPerson
Bjarne Stroustrop


In [10]:
class A:
    def __init__(self):
        print('A: {}'.format(self))
        
class B(A):
    def __init__(self):
        print('Init B')
        super().__init__()
        print('B: {}'.format(self))
        
    def fake_init(self):
        print('B fake init')
        
b = B()

Init B
A: <__main__.B object at 0x10271ad68>
B: <__main__.B object at 0x10271ad68>


### Descriptors

In [11]:
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('{} requires an argument of type {!s}'.format(self.name, self.expected_type))
        instance.__dict__[self.name] = value

In [12]:
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

In [13]:
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [14]:
vars(Stock)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Stock.__init__(self, name, shares, price)>,
              '__dict__': <attribute '__dict__' of 'Stock' objects>,
              '__weakref__': <attribute '__weakref__' of 'Stock' objects>,
              '__doc__': None,
              'name': <__main__.Typed at 0x102722048>,
              'shares': <__main__.Typed at 0x1027224e0>,
              'price': <__main__.Typed at 0x102722438>})

In [15]:
google = Stock('GOOG', 2, 1234.)
try:
    amazon = Stock('AMZN', 3, '1989')
except TypeError as err:
    print('Error: {}'.format(err))

Error: price requires an argument of type <class 'float'>


### 8.10 Lazy Properties

In [16]:
# A lazy property implemented as a descriptor
class lazyproperty:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [17]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi*(self.radius**2)

In [18]:
c = Circle(4.0)
print(vars(c))
print()
print(c.area)
print()
print(vars(c))
print()
c.area = 42
print('Area is now mutable: {}'.format(c.area))

{'radius': 4.0}

Computing area
50.26548245743669

{'radius': 4.0, 'area': 50.26548245743669}

Area is now mutable: 42


In [19]:
def immutable_lazyproperty(func):
    name = '_lazy_' + func.__name__
    
    @property
    def lazy(self):
        if hasattr(self, name):
            return getattr(self, name)
        else:
            value = func(self)
            setattr(self, name, value)
            return value
    return lazy

In [20]:
class Square:
    def __init__(self, side):
        self.side = side
        
    @immutable_lazyproperty
    def area(self):
        print('Computing area')
        return self.side**2

In [21]:
c = Square(4.0)
print(vars(c))
print()
print(c.area)
print()
print(vars(c))
print()
try:
    c.area = 42
except Exception as err:
    print('Can not arbitrary change area now')


{'side': 4.0}

Computing area
16.0

{'side': 4.0, '_lazy_area': 16.0}

Can not arbitrary change area now


### 8.11 Simplified Initialization

In [22]:
class Structure:
    _fields = []
    
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('{} expects {} arguments.'.format(self.__class__.__name__, len(self._fields)))
            
        for attr, value in zip(self._fields, args):
            setattr(self, attr, value)

In [23]:
class Circle(Structure):
    _fields = ['radius']
    
    @property
    def area(self):
        return math.pi*(self.radius**2)
    
c = Circle(7.0)
print(c.area)

153.93804002589985


In [24]:
class Stock(Structure):
    _fields = ['name', 'price', 'quantity']
    
google = Stock('GOOG', 1334., 20)
google.price

try:
    amazon = Stock('AMAZN', 1984.)
except TypeError as err:
    print('Error: {}'.format(err))

Error: Stock expects 3 arguments.


In [25]:
help(Stock)

Help on class Stock in module __main__:

class Stock(Structure)
 |  Stock(*args)
 |  
 |  Method resolution order:
 |      Stock
 |      Structure
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  _fields = ['name', 'price', 'quantity']
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Structure:
 |  
 |  __init__(self, *args)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Structure:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [26]:
def init_fromlocals(self):
    import sys
    locs = sys._getframe(1).f_locals
    for name, value in locs.items():
        if name != 'self':
            setattr(self, name, value)
            
            
class Stock:
    def __init__(self, name, shares, price):
        init_fromlocals(self)

In [27]:
google = Stock('GOOG', 1334., 20)
google.price

20

### 8.13 Type System

In [28]:
# Base descriptor
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
        
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
        

In [29]:
# Type enforcing descriptor
class Typed(Descriptor):
    expected_type = type(None)    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected value of type {}'.format(self.expected_type))
        super().__set__(instance, value)

# Value enforcing descriptor
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
            
# Size checking descriptor
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('Requires size option')
        super().__init__(name, **opts)
        
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)

In [30]:
class Integer(Typed):
    expected_type = int
    
class UnsignedInteger(Integer, Unsigned): # Uses super() to traverse both typed and unsigned __set__()
    pass

class Float(Typed):
    expected_type = float
    
class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str
    
class SizedString(String, MaxSized):
    pass

In [31]:
class Stock:
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    
    def __init__(self, name, shares, price):
        self.name  = name
        self.shares = shares
        self.price = price
        
        
google = Stock('GOOG', 25, 1340.23)
print(google.price)
try:
    hacked_stock = Stock('somerandomstock', 25, 989)
except ValueError as err:
    print('Error: {}'.format(err))

1340.23
Error: size must be < 8


#### Using a class decorator

In [32]:
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate
            

In [33]:
@check_attributes(name = SizedString(size=8),
                shares = UnsignedInteger,
                price = UnsignedFloat)
class DecoratedStock:
    def __init__(self, name, shares, price):
        self.name  = name
        self.shares = shares
        self.price = price
        
        
google = DecoratedStock('GOOG', 25, 1340.23)
print(google.price)
try:
    hacked_stock = DecoratedStock('someck', -25, 989)
except ValueError as err:
    print('Error: {}'.format(err))

1340.23
Error: Expected >= 0


#### Metaclass

In [34]:
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
        return super().__new__(cls, clsname, bases, methods)
                

In [35]:
class MetaStock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()
    def __init__(self, name, shares, price):
        self.name  = name
        self.shares = shares
        self.price = price
        
        
google = MetaStock('GOOG', 25, 1340.23)
print(google.price)
try:
    hacked_stock = MetaStock('hack', 25, 'shellscript')
except TypeError as err:
    print('Error: {}'.format(err))

1340.23
Error: Expected value of type <class 'float'>


#### Using Decorators instead of Mixins

In [36]:
def deco_typed(expected_type, cls=None):
    if cls is None:
        return lambda cls: deco_typed(expected_type, cls) # The lambda becomes the final class decorator
    
    super_set = cls.__set__ # Can not use super() since this is a decorator, not in the inheritance hierarchy
    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError('Expected value of type {}'.format(expected_type))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

def deco_maxsized(cls):
    super_init = cls.__init__
    def __init__(self, name=None, **opt):
        if 'size' not in opt:
            raise TypeError('Missing size option')
        super_init(self, name,  **opt)
    cls.__init__ = __init__
    
    super_set = cls.__set__
    def __set__(self, instance, value):
        if len(value) > self.size:
            raise ValueError('Size should be <= {}'.format(self.size))
        super_set(self, instance, value)
    cls.__set__ = __set__    
    return cls


def deco_unsigned(cls):
    super_set = cls.__set__
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Value should be >= 0')
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls    

In [37]:
@deco_typed(int)
class NewInteger(Descriptor):
    pass

@deco_typed(float)
class NewFloat(Descriptor):
    pass

@deco_typed(str)
class NewString(Descriptor):
    pass

@deco_maxsized
class NewSizedString(NewString):
    pass

@deco_unsigned
class NewUnsignedInteger(NewInteger):
    pass

@deco_unsigned
class NewUnsignedFloat(NewFloat):
    pass

In [38]:
@check_attributes(price=NewUnsignedFloat, name=NewSizedString(size=8), shares=NewUnsignedInteger)
class NewStock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
google = NewStock('GOOG', 25, 1340.23)
print(google.price)
try:
    hacked_stock = NewStock('hack', 25, 'shellscript')
    print(hacked_stock.price)
except TypeError as err:
    print('Error: {}'.format(err))
    
try:
    hacked_stock = NewStock('hacked_buffer', 25, 1340.23)
    print(hacked_stock.name)
except ValueError as err:
    print('Error: {}'.format(err))
    
try:
    hacked_stock = NewStock('AMZN', -25, 1340.23)
    print(hacked_stock.shares)
except ValueError as err:
    print('Error: {}'.format(err))

1340.23
Error: '<' not supported between instances of 'str' and 'int'
Error: Size should be <= 8
Error: Value should be >= 0


### 8.16 Multiple Constructors

In [39]:
import time

class Date:
    def __init__(self, day, month, year):
        self.day = day
        self.month = month
        self.year = year
        
    @classmethod    
    def today(cls):
        t = time.localtime()
        return cls(t.tm_mday, t.tm_mon, t.tm_year)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return '{}/{}/{}'.format(self.day, self.month, self.year)

In [40]:
a = Date(21, 12, 2012)
b = Date.today()
a, b

(21/12/2012, 18/1/2019)

In [41]:
# Create a class without __init__
d = Date.__new__(Date)
try:
    d.year
except AttributeError:
    print('Since __init__ is not called, now the user has to fill in the attributes with setattr.')

Since __init__ is not called, now the user has to fill in the attributes with setattr.


### 8.18 Mixins

In [42]:
class LoggedMappingMixin:
    # Logs getitem and setitem
    __slots__ = () # No internal state
    
    def __getitem__(self, key):
        print('Getting {}'.format(key))
        return super().__getitem__(key)
        
    def __setitem__(self, key, value):
        print('Setting ({},{}) pair'.format(key, value))
        super().__setitem__(key, value)
        

class SetOnceMappingMixin:
    # A key can only be set once
    __slots__ = () # No internal state
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError('{} already set.'.format(key))
        super().__setitem__(key, value)

In [43]:
class LoggedDict(LoggedMappingMixin, dict):
    pass

a = LoggedDict({'name': 'Bruce Wayne', 'hero': 'Batman'})
print(a['name'])
a['hero']='Caped Crusader'
print(a['hero'])

Getting name
Bruce Wayne
Setting (hero,Caped Crusader) pair
Getting hero
Caped Crusader


In [44]:
from collections import defaultdict

class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
    pass

d = SetOnceDefaultDict(list)
d['x'].append(2); 
d['y'].append(3); 
d['x'].append(4); 
d['y'].append(10);
print(dict(d))
try:
    d['x'] = 5
except KeyError as err:
    print('Error: {}'.format(err))

{'x': [2, 4], 'y': [3, 10]}
Error: 'x already set.'


### 8.19 Finite State Machine

In [45]:
# One class to rule them all
class Connection:
    def __init__(self):
        self.new_state(ClosedConnection)
        
    def new_state(self, newstate):
        self.__class__ = newstate # Directly change the class !
        
    def read(self):
        raise NotImplementedError()
        
    def write(self):
        raise NotImplementedError()
        
    def open(self):
        raise NotImplementedError()
        
    def close(self):
        raise NotImplementedError()
        
# One class for each state        
class ClosedConnection(Connection):
    def read(self):
        raise RuntimeError('Not open')
        
    def write(self):
        raise RuntimeError('Not open')
        
    def open(self):
        print('Opening')
        self.new_state(OpenConnection)
        
    def close(self):
        raise RuntimeError('Already Closed')
        

class OpenConnection(Connection):
    def read(self):
        print('reading')
        
    def write(self):
        print('writing')
        
    def open(self):
        raise RuntimeError('Already open')
        
    def close(self):
        print('Closing')
        self.new_state(ClosedConnection)

In [46]:
c = Connection()
print(c)
try:
    c.read()
except RuntimeError as err:
    print('Error: {}'.format(err))
    
c.open()
print(c)
c.read()
c.write()
c.close()
print(c)

Error: Not open
Opening
reading
writing
Closing
